In [ ]:
#Optional step to delete session if you get an error that multiple sessions are open
# del session

# Initialize Notebook, import libraries and create Snowflake connection

In [6]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, FloatType
from snowflake.snowpark.functions import avg, sum, col, udf, call_udf, call_builtin, year, month
import streamlit as st
import pandas as pd
from datetime import date

# scikit-learn (install: pip install -U scikit-learn)
from sklearn.linear_model import LinearRegression

# Session
connection_parameters = {
        "account": "******************",
        "user": "******************",
        "password": "*******************",
        "warehouse": "COMPUTE_WH",
        "role": "ACCOUNTADMIN",
        "database": "SUMMIT_HOL",
        "schema": "PUBLIC"
    }

session = Session.builder.configs(connection_parameters).create()


# test if we have a connection
session.sql("select current_account() acct, current_warehouse() wh, current_database() db, current_schema() schema, current_version() version").show()


-------------------------------------------------------------
|"ACCT"    |"WH"        |"DB"        |"SCHEMA"  |"VERSION"  |
-------------------------------------------------------------
|ODB77933  |COMPUTE_WH  |SUMMIT_HOL  |PUBLIC    |8.19.1     |
-------------------------------------------------------------



# Query the data

In [7]:
# SQL queries to explore the data

# What financial data is available as a time-series from FRED?
session.sql("SELECT DISTINCT variable_name FROM FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES").show()



------------------------------------------------------
|"VARIABLE_NAME"                                     |
------------------------------------------------------
|Market Yield on U.S. Treasury Securities at 30-...  |
|1-Year Treasury Bill Secondary Market Rate, Dis...  |
|Market Yield on U.S. Treasury Securities at 20-...  |
|30-Day AA Nonfinancial Commercial Paper Interes...  |
|Treasury Long-Term Average (Over 10 Years), Inf...  |
|Market Yield on U.S. Treasury Securities at 7-Y...  |
|Federal Funds Target Range - Upper Limit, Not s...  |
|6-Month Treasury Bill Secondary Market Rate, Di...  |
|Real Estate Loans: Commercial Real Estate Loans...  |
|Nominal Emerging Market Economies U.S. Dollar I...  |
------------------------------------------------------



In [8]:
# What is the size of all the time-series data?
session.sql("SELECT COUNT(*) FROM FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES").show()

--------------
|"COUNT(*)"  |
--------------
|2318921     |
--------------



In [9]:
# Now use Snowpark dataframe
snow_df_pce = (session.table("FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.FINANCIAL_FRED_TIMESERIES")
               .filter(col('VARIABLE_NAME') == 'Personal Consumption Expenditures: Chain-type Price Index, Seasonally adjusted, Monthly, Index 2017=100')
               .filter(col('DATE') >= '1972-01-01')
               .filter(month(col('DATE')) == 1))
pd_df_pce_year = snow_df_pce.select(year(col("DATE")).alias('"Year"'), col('VALUE').alias('PCE')).orderBy(col('DATE')).to_pandas()
pd_df_pce_year


,Year,PCE
0,1972,21.015
1,1973,21.695
2,1974,23.523
3,1975,26.132
4,1976,27.795
5,1977,29.263
6,1978,31.194
7,1979,33.597
8,1980,37.124
9,1981,41.011


# Train the Linear Regression model

In [10]:
# train model with PCE index

x = pd_df_pce_year["Year"].to_numpy().reshape(-1,1)
y = pd_df_pce_year["PCE"].to_numpy()

model = LinearRegression().fit(x, y)

# test model for 2022
predictYear = 2022
pce_pred = model.predict([[predictYear]])
# print the last 5 years
print (pd_df_pce_year.tail() )
# run the prediction for 2022
print ('Prediction for '+str(predictYear)+': '+ str(round(pce_pred[0],2)))


    Year      PCE
48  2020  104.458
49  2021  106.145
50  2022  112.829
51  2023  119.011
52  2024  121.962
Prediction for 2022: 111.72


### Creating a User Defined Function within Snowflake to do the scoring there

In [11]:
def predict_pce(predictYear: int) -> float:
    return model.predict([[predictYear]])[0].round(2).astype(float)

_ = session.udf.register(predict_pce,
                        return_type=FloatType(),
                        input_type=IntegerType(),
                        packages= ["pandas","scikit-learn"],
                        is_permanent=True, 
                        name="predict_pce_udf", 
                        replace=True,
                        stage_location="@udf_stage")

# Test the trained model by invoking the UDF via a SQL statement

In [12]:
session.sql("select predict_pce_udf(2022)").show()


---------------------------
|"PREDICT_PCE_UDF(2022)"  |
---------------------------
|111.72                   |
---------------------------

